# Flight Data Analysis

This notebook contains analysis of airline flight data from 1990-2000 using ClickHouse database.

## Setup and Connection

In [13]:
import clickhouse_connect
import pandas as pd
from datetime import datetime



client = clickhouse_connect.get_client(
    host='localhost', 
    port=8123, 
    username='default', 
    password=''
)

print("Connected to ClickHouse successfully!")

Connected to ClickHouse successfully!


## Data Quality Checks

### Describe table

In [14]:
table_info = client.query("DESCRIBE flight_data")
print("Table Schema:")
for row in table_info.result_rows:
    print(f"{row[0]}: {row[1]}")

Table Schema:
Year: Int32
Quarter: Int32
Month: Int32
DayofMonth: Int32
DayOfWeek: Int32
FlightDate: String
Reporting_Airline: String
DOT_ID_Reporting_Airline: Int32
IATA_CODE_Reporting_Airline: String
Tail_Number: String
Flight_Number_Reporting_Airline: Int32
OriginAirportID: Int32
OriginAirportSeqID: Int32
OriginCityMarketID: Int32
Origin: String
OriginCityName: String
OriginState: String
OriginStateFips: Int32
OriginStateName: String
OriginWac: Int32
DestAirportID: Int32
DestAirportSeqID: Int32
DestCityMarketID: Int32
Dest: String
DestCityName: String
DestState: String
DestStateFips: Int32
DestStateName: String
DestWac: Int32
CRSDepTime: Int32
DepTime: Float64
DepDelay: Float64
DepDelayMinutes: Float64
DepDel15: Float64
DepartureDelayGroups: Float64
DepTimeBlk: String
TaxiOut: Float64
WheelsOff: Float64
WheelsOn: Float64
TaxiIn: Float64
CRSArrTime: Int32
ArrTime: Float64
ArrDelay: Float64
ArrDelayMinutes: Float64
ArrDel15: Float64
ArrivalDelayGroups: Float64
ArrTimeBlk: String
Cance

### Check total number of records

In [26]:

total_records = client.query("SELECT COUNT(*) FROM flight_data")
print(f"Total records in database: {total_records.result_rows[0][0]:,}")

Total records in database: 58,377,437


### Check data range

In [16]:

date_range = client.query("""
    SELECT 
        MIN(Year) as min_year,
        MAX(Year) as max_year,
        COUNT(DISTINCT Year) as unique_years
    FROM flight_data
""")

min_year, max_year, unique_years = date_range.result_rows[0]
print(f"Data covers years {min_year} to {max_year} ({unique_years} unique years)")

Data covers years 1990 to 2000 (11 unique years)


### Check for missing partitions

In [17]:
partitions = client.query("""
    SELECT 
        Year,
        Month,
        COUNT(*) as record_count
    FROM flight_data
    GROUP BY Year, Month
    ORDER BY Year, Month
""")

partition_df = pd.DataFrame(partitions.result_rows, columns=['Year', 'Month', 'RecordCount'])
print("Records per year/month:")
print(partition_df)

expected_partitions = []
for year in range(1990, 2001):
    for month in range(1, 13):
        expected_partitions.append((year, month))

actual_partitions = set(zip(partition_df['Year'], partition_df['Month']))
missing_partitions = set(expected_partitions) - actual_partitions

if missing_partitions:
    print(f"\nMissing partitions: {missing_partitions}")
else:
    print("\nNo missing partitions found!")

Records per year/month:
     Year  Month  RecordCount
0    1990      1       442700
1    1990      2       396926
2    1990      3       444320
3    1990      4       433420
4    1990      5       447371
..    ...    ...          ...
127  2000      8       491366
128  2000      9       463097
129  2000     10       485761
130  2000     11       467251
131  2000     12       481038

[132 rows x 3 columns]

No missing partitions found!


## Analysis Questions

### 1. On which day were there the most flights?

In [18]:
most_flights_day = client.query("""
    SELECT 
        Year,
        Month,
        DayofMonth,
        COUNT(*) as flight_count
    FROM flight_data
    GROUP BY Year, Month, DayofMonth
    ORDER BY flight_count DESC
    LIMIT 1
""")

year, month, day, count = most_flights_day.result_rows[0]
print(f"Day with most flights: {year}-{month:02d}-{day:02d} with {count:,} flights")

Day with most flights: 2000-12-18 with 16,218 flights


### 2. Which day of the week had the fewest flights in 1995?

In [19]:
fewest_flights_dow_1995 = client.query("""
    SELECT 
        DayOfWeek,
        COUNT(*) as flight_count
    FROM flight_data
    WHERE Year = 1995
    GROUP BY DayOfWeek
    ORDER BY flight_count ASC
    LIMIT 1
""")

dow, count = fewest_flights_dow_1995.result_rows[0]
day_names = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
print(f"Day of week with fewest flights in 1995: {day_names.get(dow, dow)} with {count:,} flights")

Day of week with fewest flights in 1995: Saturday with 695,286 flights


### 3. Number of flights delayed by more than 10 minutes, grouped by day of the week

In [20]:
delays_by_dow = client.query("""
    SELECT 
        DayOfWeek,
        COUNT(*) as delayed_flights
    FROM flight_data
    WHERE DepDelay > 10
    GROUP BY DayOfWeek
    ORDER BY DayOfWeek
""")

print("Flights delayed by more than 10 minutes by day of week:")
for dow, count in delays_by_dow.result_rows:
    print(f"{day_names.get(dow, dow)}: {count:,} flights")

Flights delayed by more than 10 minutes by day of week:
Monday: 1,419,299 flights
Tuesday: 1,349,905 flights
Wednesday: 1,504,265 flights
Thursday: 1,744,321 flights
Friday: 1,898,026 flights
Saturday: 1,293,669 flights
Sunday: 1,530,053 flights


### 4. Number of delays by carrier for 1997

In [21]:
delays_by_carrier_1997 = client.query("""
    SELECT 
        Reporting_Airline as UniqueCarrier,
        COUNT(*) as delayed_flights
    FROM flight_data
    WHERE Year = 1997 AND DepDelay > 0
    GROUP BY UniqueCarrier
    ORDER BY delayed_flights DESC
""")

print("Number of delays by carrier for 1997:")
for carrier, count in delays_by_carrier_1997.result_rows:
    print(f"{carrier}: {count:,} delayed flights")

Number of delays by carrier for 1997:
DL: 481,009 delayed flights
UA: 412,867 delayed flights
US: 317,819 delayed flights
WN: 290,235 delayed flights
AA: 246,490 delayed flights
NW: 212,544 delayed flights
CO: 169,227 delayed flights
TW: 82,727 delayed flights
HP: 64,982 delayed flights
AS: 49,442 delayed flights


### 5. Percentage of delays by carrier for 1997

In [22]:
delay_percentage_by_carrier_1997 = client.query("""
    SELECT 
        Reporting_Airline as UniqueCarrier,
        COUNT(*) as total_flights,
        SUM(CASE WHEN DepDelay > 0 THEN 1 ELSE 0 END) as delayed_flights,
        (SUM(CASE WHEN DepDelay > 0 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as delay_percentage
    FROM flight_data
    WHERE Year = 1997
    GROUP BY UniqueCarrier
    ORDER BY delay_percentage DESC
""")

print("Percentage of delays by carrier for 1997:")
for carrier, total, delayed, percentage in delay_percentage_by_carrier_1997.result_rows:
    print(f"{carrier}: {percentage:.2f}% ({delayed:,} out of {total:,} flights)")

Percentage of delays by carrier for 1997:
UA: 55.50% (412,867 out of 743,847 flights)
DL: 52.18% (481,009 out of 921,850 flights)
US: 44.22% (317,819 out of 718,751 flights)
CO: 41.96% (169,227 out of 403,349 flights)
NW: 39.57% (212,544 out of 537,152 flights)
AA: 37.12% (246,490 out of 663,954 flights)
WN: 36.51% (290,235 out of 794,849 flights)
AS: 33.46% (49,442 out of 147,750 flights)
HP: 31.50% (64,982 out of 206,304 flights)
TW: 30.19% (82,727 out of 274,037 flights)


### 6. Percentage of flights delayed for more than 10 minutes, grouped by year

In [23]:
delay_percentage_by_year = client.query("""
    SELECT 
        Year,
        COUNT(*) as total_flights,
        SUM(CASE WHEN DepDelay > 10 THEN 1 ELSE 0 END) as delayed_flights,
        (SUM(CASE WHEN DepDelay > 10 THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as delay_percentage
    FROM flight_data
    GROUP BY Year
    ORDER BY Year
""")

print("Percentage of flights delayed for more than 10 minutes by year:")
for year, total, delayed, percentage in delay_percentage_by_year.result_rows:
    print(f"{year}: {percentage:.2f}% ({delayed:,} out of {total:,} flights)")

Percentage of flights delayed for more than 10 minutes by year:
1990: 16.65% (877,347 out of 5,270,893 flights)
1991: 14.72% (747,406 out of 5,076,925 flights)
1992: 14.68% (747,296 out of 5,092,157 flights)
1993: 15.42% (782,124 out of 5,070,501 flights)
1994: 16.57% (858,232 out of 5,180,048 flights)
1995: 19.39% (1,033,173 out of 5,327,435 flights)
1996: 22.18% (1,187,220 out of 5,351,983 flights)
1997: 19.17% (1,037,187 out of 5,411,843 flights)
1998: 19.36% (1,042,287 out of 5,384,721 flights)
1999: 20.09% (1,110,409 out of 5,527,884 flights)
2000: 23.17% (1,316,857 out of 5,683,047 flights)


### 7. Top 10 most popular destinations

In [24]:
top_destinations = client.query("""
    SELECT 
        Dest,
        COUNT(*) as flight_count
    FROM flight_data
    GROUP BY Dest
    ORDER BY flight_count DESC
    LIMIT 10
""")

print("Top 10 most popular destinations:")
for i, (dest, count) in enumerate(top_destinations.result_rows, 1):
    print(f"{i}. {dest}: {count:,} flights")

Top 10 most popular destinations:
1. ORD: 3,179,174 flights
2. DFW: 2,781,284 flights
3. ATL: 2,610,247 flights
4. LAX: 1,926,312 flights
5. PHX: 1,715,468 flights
6. STL: 1,712,218 flights
7. DTW: 1,558,308 flights
8. DEN: 1,540,166 flights
9. SFO: 1,422,341 flights
10. MSP: 1,418,363 flights
